Получаем вектора и кластеры

In [196]:
import pandas as pd
import json
import re
import random
from tqdm import tqdm
from sklearn.cluster import SpectralClustering, KMeans
from sklearn import metrics
import pickle

from wordcloud import WordCloud
from stop_words import get_stop_words

from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
#Предобученный rubert от DeepPavlov
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

In [2]:
def embed_bert_cls(text:str):
    """
    Получаем эмбеддинг текста
    """
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt', max_length=5000)
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [4]:
def text_processing(txt: str) -> str:
    return txt.replace("\xa0", " ")

In [154]:
data = pd.read_json("../data/corpus.json")
data['full_text'] = data['title'].str.cat(data['annotation'], sep='\n').apply(text_processing)

In [20]:
embds = []
ids=[]
for i, article in enumerate(data['full_text'].values):
    try:
        embd = embed_bert_cls(article)
        embds.append(embd)
    except:
        ids.append(i)
data['embeddinigs'] = embds

100%|███████████████████████████████████████| 6000/6000 [21:42<00:00,  4.61it/s]


In [172]:
# попробовал несколько подходов к кластеризации
# по данным остановился на 4 кластерах. Только не удалось вытащить Covid и вакцины в отдельный кластер, но тут косяк в DeepPavlov
# это можно было бы решить, поработав со словами, убрав лишние, например

clustering = KMeans(n_clusters=4, algorithm="lloyd", n_init=20, max_iter=500).fit(embds)
#clustering = SpectralClustering(n_clusters=3, assign_labels='discretize', eigen_solver="lobpcg").fit(embds)

In [173]:
# сохраняем эмбеддинги и кластеры
data['cluster'] = clustering.labels_
data.to_json("../data/dataset.json")

In [210]:
# сохраняем модель
with open("../data/cluster.pkl", "wb") as f:
    pickle.dump(clustering, f)

Генерация облаков слов

In [212]:
model 

KMeans(max_iter=500, n_clusters=4, n_init=20)

In [ ]:
# сохраняем модель
with open("../data/cluster.pkl", "wb") as f:
    pickle.dump(clustering, f)

In [174]:
data = pd.read_json("../data/dataset.json")

In [175]:
for k, d in data.groupby("cluster"):
    print(len(d))

239
1873
2087
276


In [190]:
# генератор облака слов + убираем союзы, местоимения
# из облака тегов убрал упоминания москвы, так как это все московские новости
stopwords = get_stop_words('ru') + ['москва', 'москве', 'москвы']
wordcloud = WordCloud(stopwords=stopwords, width=1920,
                      height=1080, background_color="white")

In [211]:
# этот процесс был вручную
# смотрел на облако и отмечал названия для кластера

In [ ]:
texts = "\n".join(list(data.groupby("cluster"))[3][1].full_text.values)
wc = wordcloud.generate(texts.lower())
wc.to_image()

In [189]:
wc.to_file("../images/3_ecology.png")